In [87]:
%%javascript

IPython.keyboard_manager.command_shortcuts.add_shortcut('Ctrl-k', {
    help : 'move up selected cells',
    help_index : 'jupyter-notebook:move-selection-up',
    handler : function (event) {
        IPython.notebook.move_selection_up();
        return false;
    }}
);

IPython.keyboard_manager.command_shortcuts.add_shortcut('Ctrl-j', {
    help : 'move down selected cells',
    help_index : 'jupyter-notebook:move-selection-down',
    handler :  function (event) {
        IPython.notebook.move_selection_down();
        return false;
    }}
);

<IPython.core.display.Javascript object>

In [88]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from scipy.sparse import hstack
from xgboost.sklearn import XGBClassifier # <3
from sklearn.model_selection import train_test_split
import gc
import matplotlib.pyplot as plt

from scipy.sparse.csr import csr_matrix

pd.options.mode.chained_assignment = None


In [89]:

pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 200)


In [90]:
df = pd.read_csv('events_up_to_01062018.csv', low_memory = False)
df_labels= pd.read_csv('labels_training_set.csv', low_memory = False)


In [91]:
df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
df_model = df.loc[df['model'].notnull()]
df_model = df_model.groupby('person')['model'].apply(list).reset_index()

In [93]:
#drop nans
df_model.head()
df_model.shape

(38256, 2)

In [94]:
#df_all = df_labels.merge(df_model, left_on = 'person', right_on = 'person', how = 'left')
#df_all.head()

In [95]:
df_model['model'] = df_model['model'].apply(lambda x: ', '.join(map(str, x)))
df_model = df_model.fillna('')
df_model.shape

(38256, 2)

In [96]:
X,y= df_all.iloc[:,2],df_all.iloc[:,1]

In [97]:
word_vectorizer = TfidfVectorizer()

tf_mat = word_vectorizer.fit_transform(df_model['model'])

tf_array = tf_mat.toarray()
tf_array.shape

(38256, 150)

In [98]:
doc = 0
feature_names = word_vectorizer.get_feature_names()
feature_index = tf_mat[doc,:].nonzero()[1]
tfidf_scores = zip(feature_index, [tf_mat[doc, x] for x in feature_index])

In [99]:
for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
  print (w, s)

lg 0.3679161312031975
g4 0.3631764932418822
h818p 0.5645997432092993
samsung 0.16627214468089802
galaxy 0.1663237685137348
s6 0.2985617687187759
flat 0.3521990105086365
iphone 0.17184772309950505
se 0.34050367160629463


In [100]:
feature_names

['10',
 '15',
 '16',
 '2016',
 '2017',
 '32',
 '3g',
 '4g',
 '4s',
 '5c',
 '5s',
 '6s',
 'a3',
 'a5',
 'a7',
 'a7010',
 'a9',
 'air',
 'aqua',
 'asus',
 'beat',
 'chip',
 'classic',
 'com',
 'compact',
 'core',
 'd337',
 'd618',
 'd690',
 'd724',
 'd821',
 'd855',
 'deluxe',
 'dtv',
 'dual',
 'duos',
 'e1',
 'e2',
 'e4',
 'e5',
 'e7',
 'edge',
 'edition',
 'fi',
 'fit',
 'flat',
 'force',
 'frontier',
 'g1',
 'g2',
 'g3',
 'g4',
 'g5',
 'g5s',
 'galaxy',
 'gb',
 'gear',
 'go',
 'gran',
 'grand',
 'grande',
 'h522',
 'h540t',
 'h630',
 'h736',
 'h815p',
 'h818p',
 'hdtv',
 'i9082',
 'i9300',
 'i9300i',
 'i9500',
 'i9505',
 'i9515',
 'ipad',
 'iphone',
 'j1',
 'j2',
 'j3',
 'j5',
 'j7',
 'k10',
 'k4',
 'k5',
 'k8',
 'l80',
 'laser',
 'led',
 'lenovo',
 'lg',
 'live',
 'm4',
 'max',
 'maxx',
 'mega',
 'metal',
 'mini',
 'moto',
 'motorola',
 'muv',
 'n7100',
 'neo',
 'new',
 'nexus',
 'note',
 'novo',
 'on',
 'outros',
 'pen',
 'pequeno',
 'play',
 'plus',
 'pocket',
 'power',
 'premium',

In [101]:
df_tfidf = pd.DataFrame(tf_array, columns=feature_names)

In [102]:

df_tfidf['person'] = df_model['person']
df_tfidf.head()

,10,15,16,2016,2017,32,3g,4g,4s,5c,5s,6s,a3,a5,a7,a7010,a9,air,aqua,asus,beat,chip,classic,com,compact,...,samsung,screen,se,selfie,slim,sony,style,stylus,tab,tv,ultra,up,vibe,wi,win,x2,xiaomi,xperia,you,young,z2,z3,z5,zenfone,person
0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.166272,0.0,0.340504,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0008ed71
1,0.0,0.0,0.0,0.015199,0.037607,0.0,0.000000,0.012888,0.0,0.00000,0.000000,0.552197,0.000000,0.010325,0.026251,0.0,0.01566,0.007388,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.143194,0.0,0.019229,0.0,0.000,0.005728,0.235084,0.0,0.0,0.000000,0.0,0.0,0.006189,0.0065,0.0,0.063613,0.0,0.005728,0.0,0.0,0.0,0.006312,0.0,0.0,00091926
2,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.395531,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.506176,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,00091a7a
3,0.0,0.0,0.0,0.173834,0.058652,0.0,0.026908,0.194300,0.0,0.05365,0.006084,0.000000,0.509484,0.177137,0.040942,0.0,0.00000,0.000000,0.011998,0.0,0.0,0.0,0.0,0.0,0.0,...,0.402728,0.0,0.000000,0.0,0.064,0.008933,0.000000,0.0,0.0,0.049939,0.0,0.0,0.000000,0.0000,0.0,0.079371,0.0,0.008933,0.0,0.0,0.0,0.000000,0.0,0.0,000ba417
4,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,000c79fe


## XgBoost

In [103]:
df_train = df_labels.merge(df_tfidf , left_on='person', right_on='person', how='left')

In [104]:
df_train.head()

,person,label,10,15,16,2016,2017,32,3g,4g,4s,5c,5s,6s,a3,a5,a7,a7010,a9,air,aqua,asus,beat,chip,classic,...,s8,samsung,screen,se,selfie,slim,sony,style,stylus,tab,tv,ultra,up,vibe,wi,win,x2,xiaomi,xperia,you,young,z2,z3,z5,zenfone
0,0566e9c1,0,0.0,0.0,0.0,0.067941,0.0,0.0,0.0,0.05761,0.507094,0.0,0.209243,0.0,0.000000,0.00000,0.070408,0.0,0.0,0.0,0.000000,0.0,0.107296,0.0,0.0,...,0.0,0.346286,0.0,0.064468,0.0,0.0,0.076814,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.076814,0.0,0.0,0.0,0.084648,0.0,0.0
1,6ec7ee77,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,abe7a2fb,0,0.0,0.0,0.0,0.119215,0.0,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.18222,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.386666,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3,34728364,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.070456,0.00000,0.000000,0.0,0.0,0.0,0.228973,0.0,0.000000,0.0,0.0,...,0.0,0.232896,0.0,0.000000,0.0,0.0,0.170484,0.0,0.0,0.0,0.0,0.0,0.0,0.368431,0.0,0.0,0.126225,0.0,0.170484,0.0,0.0,0.0,0.000000,0.0,0.0
4,87ed62de,0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.421100,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [105]:
X, y = df_train.iloc[:,2:],df_train.iloc[:,1]
X.head()

,10,15,16,2016,2017,32,3g,4g,4s,5c,5s,6s,a3,a5,a7,a7010,a9,air,aqua,asus,beat,chip,classic,com,compact,...,s8,samsung,screen,se,selfie,slim,sony,style,stylus,tab,tv,ultra,up,vibe,wi,win,x2,xiaomi,xperia,you,young,z2,z3,z5,zenfone
0,0.0,0.0,0.0,0.067941,0.0,0.0,0.0,0.05761,0.507094,0.0,0.209243,0.0,0.000000,0.00000,0.070408,0.0,0.0,0.0,0.000000,0.0,0.107296,0.0,0.0,0.0,0.0,...,0.0,0.346286,0.0,0.064468,0.0,0.0,0.076814,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.076814,0.0,0.0,0.0,0.084648,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.119215,0.0,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.18222,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.386666,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.070456,0.00000,0.000000,0.0,0.0,0.0,0.228973,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.232896,0.0,0.000000,0.0,0.0,0.170484,0.0,0.0,0.0,0.0,0.0,0.0,0.368431,0.0,0.0,0.126225,0.0,0.170484,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.421100,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [106]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [107]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [108]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 4,
                subsample = 0.8,
                gamma = 1,
                n_estimators = 150) #150 = 0.61730  #1500 = 0.61296 , 

In [109]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)


In [110]:
xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=150,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [111]:
preds = xg_reg.predict(X_test)

In [112]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.215485


In [113]:
persons = df_labels['person']
df_predict = df_tfidf.loc[~df_tfidf.person.isin(persons)]
ppl_to_predict = (df.loc[~df['person'].isin(persons)])['person'].to_frame()
ppl_to_predict = ppl_to_predict.drop_duplicates('person')
df_predict = ppl_to_predict.merge(df_predict, left_on = 'person' , right_on = 'person', how='left')
X_predict = df_predict.drop(['person'], axis=1)

In [114]:
entrie = xg_reg.predict(X_predict)

In [115]:
seriesita = pd.Series(entrie)

In [116]:
df_entrie = df_predict['person'].to_frame()
df_entrie['label'] = seriesita

In [117]:
df_entrie.head()

,person,label
0,4886f805,0.027665
1,0297fc1e,0.049354
2,2d681dd8,0.046165
3,cccea85e,0.059765
4,4c8a8b93,0.029902


In [118]:
df_entrie = df_entrie.fillna(0)
df_entrie['label'].value_counts()

0.027665    4187
0.046165    1441
0.036783    1049
0.030624     475
0.036895     471
0.119648     289
0.031109     239
0.030853     188
0.049752     179
0.035383     177
0.055283     154
0.025764     154
0.044264     150
0.031795     143
0.049124     142
0.031803     141
0.050303     129
0.031129     127
0.040447     107
0.034297     101
0.044500      95
0.037017      84
0.029618      80
0.034460      78
0.031252      76
0.039384      66
0.049609      63
0.032380      57
0.025635      56
0.075324      53
0.039521      48
0.049354      46
0.030856      45
0.041075      42
0.034051      39
0.039741      37
0.049564      37
0.047724      34
0.022705      33
0.041479      31
0.029540      29
0.039971      29
0.043618      26
0.030319      25
0.043865      25
0.033829      25
0.030927      24
0.052940      24
0.041689      24
0.029221      24
0.050296      23
0.042003      23
0.044585      22
0.033748      22
0.053883      22
0.043661      21
0.020996      21
0.047223      21
0.048119      

In [119]:
num = df_entrie._get_numeric_data()
num[num < 0] = 0
df_entrie['label'].value_counts()


0.027665    4187
0.046165    1441
0.036783    1049
0.030624     475
0.036895     471
0.119648     289
0.031109     239
0.030853     188
0.049752     179
0.035383     177
0.055283     154
0.025764     154
0.044264     150
0.031795     143
0.049124     142
0.031803     141
0.050303     129
0.031129     127
0.040447     107
0.034297     101
0.044500      95
0.037017      84
0.029618      80
0.034460      78
0.031252      76
0.039384      66
0.049609      63
0.032380      57
0.025635      56
0.075324      53
0.039521      48
0.049354      46
0.030856      45
0.041075      42
0.034051      39
0.039741      37
0.049564      37
0.047724      34
0.022705      33
0.041479      31
0.029540      29
0.039971      29
0.043618      26
0.030319      25
0.043865      25
0.033829      25
0.030927      24
0.052940      24
0.041689      24
0.029221      24
0.050296      23
0.042003      23
0.044585      22
0.033748      22
0.053883      22
0.043661      21
0.020996      21
0.047223      21
0.048119      

In [120]:
df_entrie.to_csv(path_or_buf = 'entrie2.0', index = False)

In [121]:
df_entrie.shape

(19415, 2)

In [122]:
df_entrie['label'].nlargest(1)

254    0.568125
Name: label, dtype: float32

In [123]:
a

NameError: name 'a' is not defined